In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import os
from tqdm import tqdm_notebook as tqdm
#from codecs import open
from lxml import etree

from nlppln.utils import out_file_name, create_dirs

In [ ]:
with open('/home/jvdzwaan/data/dbnl_ocr/raw/ocr-with-title-page/_aio001jver01_01.txt') as f:
    ls = f.readlines()

with open('/home/jvdzwaan/data/dbnl_ocr/raw/notes/_aio001jver01_01.txt') as f:
    notes = f.readlines()

print(len(ls))
print(len(notes))

In [ ]:
# remove empty lines from lines
lines = []

for line in ls:
    if line.strip() != '':
        lines.append(line)
print(len(lines))

In [ ]:
lines = ls

In [ ]:
lines[3239]

In [ ]:
# filter notes shorter than threshold
t = 6

notes = [n for n in filter(lambda s: len(s.strip())>t, notes)]
print(len(notes))

In [ ]:
# unique notes with the repeated notes
print(len(set(notes)))

In [ ]:
# identieke notes die meerdere keren voorkomen
from collections import Counter

c = Counter()
num_lines = 0

for note in notes:
    c[note] += 1

repeated = []
ns = []
for k, v in c.most_common():
    if v > 1:
        num_lines += v
        print(repr(k), v)
        repeated.append(k)
    else:
        ns.append(k)
print('total number of lines:', num_lines)

In [ ]:
# unique notes without the repeated notes
print(len(ns))

In [ ]:
%%time
# Calculate edit distance between repeated notes and lines
import edlib

# get initial edit distances per line
line_data = pd.DataFrame()
for i, n in enumerate(repeated):
    line_data[i] = [edlib.align(n, line)['editDistance'] for line in lines]

In [ ]:
%%time
# delete lines that have sufficient overlap with note text

def get_lines(column, t):
    #print(list(column[column < t].index))
    return list(column[column < t].index)

t = 0.3

lengths = [len(x) for x in lines]
ld = line_data.apply(lambda c: c/lengths)

In [ ]:
ld

In [ ]:
c = 0
t = 0.4
ld[ld[c] < t][c]

In [ ]:
to_remove = [ids for ids in ld.apply(lambda c: get_lines(c, t))]
to_remove = list(set([item for sublist in to_remove for item in sublist]))
print(len(to_remove))

In [ ]:
to_remove.sort()
print(to_remove)

In [ ]:
ls = lines
lines = []
for i, line in enumerate(ls):
    if i not in to_remove:
        lines.append(line)
print(len(lines))

In [ ]:
%%time
# Calculate edit distance between remaining notes and lines
import edlib

notes = ns

# get initial edit distances per line
line_data = pd.DataFrame()
for i, n in enumerate(notes):
    line_data[i] = [edlib.align(n, line)['editDistance'] for line in lines]

In [ ]:
line_data

In [ ]:
%%time
import math
import copy
import edlib

ns = [{'note': n, 'ed': line_data[i].min(), 'fragment': lines[i], 'l': 1, 
       'lines': line_data[i].sort_values().index[:100]} for i, n in enumerate(notes)]
l = 2
num_changed = len(ns)

In [ ]:
def ngrams(tokens, n=3):
    ngrams = zip(*[tokens[i:] for i in range(n)])  
    return [ngram for ngram in ngrams]

ngrams(['1', '2', '3', '4', '5', '6'])

In [ ]:
%%time
import math
import copy
import edlib

l = 2
num_changed = len(ns)

while num_changed >= 1:
    num_changed = 0
    num_checked = 0
    print(l)
    for n in ns:
        if n['l']+1 >= l:
            num_checked += 1
            note = n['note']
    
            for ngram in ngrams(range(len(lines)), n=l):
                test = False
                for lineno in ngram:
                    if lineno in n['lines']:
                        test = True
                if test:
                    #print(ngram)
                    #print(n)
                    #print(txt)
                    txt = ''.join([lines[i] for i in ngram])
                    res = edlib.align(note, txt)
                    ed = res['editDistance']
                    #print(ed)
                    if ed < n['ed']:
                        n['ed'] = ed
                        n['fragment'] = txt
                        n['l'] = l
                        num_changed += 1
                        
                    #print(ngram)
                    #print(res['editDistance'])
                    #print('---')

            #import sys
            #sys.exit()
            #print('note:', note)
            #print('fragment:', fragment)
            #print('editdistance:', min_val)
            #print('-----')
    print('num checked', num_checked)
    print('num changed', num_changed)
    l += 1

In [ ]:
lines.index('1) Hs. u met eene o er boven. \n')

In [ ]:
for n in ns:
    if n['l'] == 27:
        print(n['note'])
        print('---')
        print(n['fragment'])
        print('---')
        print(n['ed'])
        print(n['l'])
        print(n['lines'])
        print('-----')

In [ ]:
for n in ns:
    if n['ed'] > 200:
        print(n['note'])
        print('---')
        print(n['fragment'])
        print('---')
        print(n['ed'])
        print(n['l'])
        print('-----')

In [ ]:
eds = [n['ed'] for n in ns]
#print(eds)
plt.figure(figsize=(15,10))
plt.hist(eds, 100)

In [ ]:
# identieke notes die meerdere keren voorkomen
from collections import Counter

c = Counter()
num_lines = 0

for note in notes:
    c[note] += 1

repeated = []
ns = []
for k, v in c.most_common():
    if v > 1:
        num_lines += v
        print(repr(k), v)
        repeated.append(k)
    else:
        ns.append(k)
print('total number of lines:', num_lines)

In [ ]:
%%time
# Calculate overlap between repeated notes and lines
from fuzzywuzzy import fuzz

line_data = pd.DataFrame()
for i, n in enumerate(repeated):
    line_data[i] = [fuzz.partial_ratio(line, n) for line in lines]

In [ ]:
line_data

In [ ]:
for i in line_data.columns:
    print(i)
    print(line_data[line_data[i].sort_values(ascending=False) > 90].shape)

In [ ]:
def get_lines(column, t):
    #print(list(column[column < t].index))
    return list(column[column > t].index)

t = 90

to_remove = line_data.apply(lambda c: get_lines(c, t))
print(to_remove)

In [ ]:
to_remove = list(set([item for sublist in to_remove for item in sublist]))
print(len(to_remove))

In [ ]:
print(len(lines))

In [ ]:
ls = lines
lines = []
for i, line in enumerate(ls):
    if i not in to_remove:
        lines.append(line)
print(len(lines))

In [ ]:
%%time
# Calculate overlap between remaining notes and lines
from fuzzywuzzy import fuzz

line_data = pd.DataFrame()
for i, n in enumerate(notes):
    line_data[i] = [fuzz.partial_ratio(line, n) for line in lines]

In [ ]:
%%time
# Calculate overlap between remaining notes and lines
from py_stringmatching.similarity_measure.partial_ratio import PartialRatio
pr = PartialRatio()

line_data = pd.DataFrame()
for i, n in enumerate(notes):
    line_data[i] = [pr.get_raw_score(line, n) for line in lines]

In [ ]:
%%time
# Calculate edit distance between remaining notes and lines
import edlib

# get initial edit distances per line
line_data = pd.DataFrame()
for i, n in enumerate(notes):
    line_data[i] = [edlib.align(n, line)['editDistance'] for line in lines]

In [ ]:
def extend_lines(indexes, length, num=3):
    #print('start')
    #print(indexes)
    #print('num:', num)
    result = []
    for i in indexes:
        #print('i', i)
        result.append(i)
        # lines before i
        start = max(0, i-num)      
        to_add = list(range(start, i))
        for n in to_add:
            result.append(n)
        #print(to_add)
        
        # lines after i
        end = min(length, i+num+1)
        #print('end', end)
        to_add = list(range(i, end))
        for n in to_add:
            result.append(n)
        #print(to_add)
        #print('---')
            
    result = list(set(result))
    result.sort()
            
    return(result)
        
extend_lines([5], 6)

In [ ]:
list(range(4,3))

In [ ]:
l = list(line_data[0].sort_values().index[:100])
r = extend_lines(l, len(lines))
print(len(l), len(r))

In [ ]:
%%time
ns = [{'note': n, 'lines': extend_lines(list(line_data[i].sort_values().index[:100]), len(lines)), 'selected': [], 'scores': []} 
          for i, n in enumerate(notes)]

In [ ]:
for n in ns:
    print(len(n['lines']))

In [ ]:
%%time

for n in ns:
    ls = n['lines']
    ls.sort()
    note = n['note']
    for idx in ls:
        r = fuzz.partial_ratio(lines[idx], note)
        n['scores'].append(r)
        
        if r > 90:
            n['selected'].append(idx)

In [ ]:
added = 0
for i, note in enumerate(ns):
    #print(note['selected'])
    #print(note['lines'])

    if note['selected'] != []:
        n = note['selected'][-1] + 1
        add = range(n, n+3)
        for a in add:
            if a in note['lines']:
                idx = note['lines'].index(a)
                #print(idx)
                #print(note['scores'][idx], lines[add])
                #print(note['scores'][idx+1], lines[add+1])
                #print(note['scores'][idx+2], lines[add+2])
                if note['scores'][idx] > 80:
                    note['selected'].append(a)
                    print(i)
                    added += 1
print(added)

In [ ]:
ns[173]

In [ ]:
lines[2456]

In [ ]:
for i, n in enumerate(ns):
    print(i)
    print(n['note'])
    print('---')
    print(n['selected'])
    for idx in n['selected']:
        print(lines[idx])
    print('-'*80)

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(ns[11]['lines'][:100], ns[11]['scores'][:100], 100)

In [ ]:
note = ns[11]
#print(note['selected'])
#print(note['lines'])

add = note['selected'][-1] + 1
if add in note['lines']:
    idx = note['lines'].index(add)
    #print(idx)
    #print(note['scores'][idx], lines[add])
    #print(note['scores'][idx+1], lines[add+1])
    #print(note['scores'][idx+2], lines[add+2])
    if note['scores'][idx] > 80:
        note['selected'].append(add)

In [ ]:
print(note['selected'])

In [ ]:
fuzz.partial_ratio('hooren van Aiols naam (Fransche tekst vs. 7190 vlgg.). ', 
                   'hooren van Aiols naam (Fransche tekst vs. 7190 vlgg.).')

In [ ]:
fuzz.partial_ratio('side. - 179 Hs. IIII.', 
                   'side. — 179 GIs. IIII.')

In [ ]:
%%time
from nlppln.utils import create_dirs, out_file_name

in_file = '/home/jvdzwaan/data/dbnl_ocr/raw/ocr-with-title-page/_aio001jver01_01.txt'

# remove selected lines
with open(in_file) as f:
    text = f.read()

for n in ns:
    for idx in n['selected']:
        #print(idx)
        l = lines[idx]
        
        text = text.replace(l, '')

out_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/ocr'
create_dirs(out_dir)
out = out_file_name(out_dir, in_file)
print(out)
with open(out, 'w') as f:
    f.write(text)

In [ ]:
print(to_remove)

In [ ]:
lines[43]

In [ ]:
%%time
import edlib

import pandas as pd

from collections import Counter
from fuzzywuzzy import fuzz

def split_notes(notes):
    c = Counter()
    num_lines = 0

    for note in notes:
        c[note] += 1

    repeated = []
    ns = []
    for k, v in c.most_common():
        if v > 1:
            num_lines += v
            #print(repr(k), v)
            repeated.append(k)
        else:
            ns.append(k)
    #print('total number of lines:', num_lines)
    
    return ns, repeated

def get_lines(column, threshold):
    return list(column[column > threshold].index)


def extend_lines(indexes, length, num=3):
    #print('start')
    #print(indexes)
    #print('num:', num)
    result = []
    for i in indexes:
        #print('i', i)
        result.append(i)
        # lines before i
        start = max(0, i-num)      
        to_add = list(range(start, i))
        for n in to_add:
            result.append(n)
        #print(to_add)
        
        # lines after i
        end = min(length, i+num+1)
        #print('end', end)
        to_add = list(range(i, end))
        for n in to_add:
            result.append(n)
        #print(to_add)
        #print('---')
            
    result = list(set(result))
    result.sort()
            
    return(result)


def remove_notes(ocr_file, notes_file, out_dir):
    with open(ocr_file) as f:
        ls = f.readlines()

    with open(notes_file) as f:
        notes = f.readlines()
        
    # remove empty lines
    lines = []

    for line in ls:
        if line.strip() != '':
            lines.append(line)
    #print(len(lines))
    
    # get repeated notes
    ns, repeated = split_notes(notes)
    
    print('processing repeated notes')

    # Calculate overlap between repeated notes and lines
    line_data = pd.DataFrame()
    for i, n in enumerate(repeated):
        line_data[i] = [fuzz.partial_ratio(line, n) for line in lines]

    # get the line numbers of the repeated notes that should be removed
    t = 90
    to_remove_repeated = line_data.apply(lambda c: get_lines(c, t))
    to_remove_repeated = list(set([item for sublist in to_remove_repeated for item in sublist]))

    print('Processing other notes')
    
    # get initial edit distances per line
    # uses edlib for speed
    print('Calculating distances with edlib')
    line_data = pd.DataFrame()
    for i, n in enumerate(notes):
        line_data[i] = [edlib.align(n, line)['editDistance'] for line in lines]
    
    # select the top 100 lines with smallest edit distances for further processing 
    ns = [{'note': n, 
           'lines': extend_lines(list(line_data[i].sort_values().index[:100]), len(lines)), 
           'selected': [], 
           'scores': []} 
          for i, n in enumerate(notes)]
    
    print('Calculating distances with fuzzywuzzy')
    # use partial_ratio to select the lines that should be deleted
    for n in ns:
        ls = n['lines']
        ls.sort()
        note = n['note']
        for idx in ls:
            r = fuzz.partial_ratio(lines[idx], note)
            n['scores'].append(r)
        
            if r > 90:
                n['selected'].append(idx)
    
    print('Adding missing lines')
    # add missing (usually short) lines at the end of selected pieces of text
    added = 0
    for i, note in enumerate(ns):
        #print(note['selected'])
        #print(note['lines'])

        if note['selected'] != []:
            n = note['selected'][-1] + 1
            add = range(n, n+3)
            for a in add:
                if a in note['lines']:
                    idx = note['lines'].index(a)
                    #print(idx)
                    #print(note['scores'][idx], lines[add])
                    #print(note['scores'][idx+1], lines[add+1])
                    #print(note['scores'][idx+2], lines[add+2])
                    if note['scores'][idx] > 80:
                        note['selected'].append(a)
                        #print(i)
                        added += 1
    if added > 0:
        print('{} lines added to be removed.'.format(added))
    
    print('Removing notes')
    # get the ocr text
    with open(ocr_file) as f:
        text = f.read()
        
    for idx in to_remove_repeated:
        l = lines[idx]
        text = text.replace(l, '')

    for n in ns:
        for idx in n['selected']:
            #print(idx)
            l = lines[idx]
            text = text.replace(l, '')

    # save result
    create_dirs(out_dir)
    out = out_file_name(out_dir, ocr_file)
    #print(out)
    with open(out, 'w') as f:
        f.write(text)

#remove_notes('/home/jvdzwaan/data/dbnl_ocr/raw/ocr-with-title-page/_aio001jver01_01.txt', 
#             '/home/jvdzwaan/data/dbnl_ocr/raw/notes/_aio001jver01_01.txt',
#             '/home/jvdzwaan/data/dbnl_ocr/raw/ocr')

In [ ]:
import os

from tqdm import tqdm_notebook as tqdm

from nlppln.utils import get_files, out_file_name

in_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/ocr-without-title-page/'
notes_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/notes/'
out_dir = '/home/jvdzwaan/data/dbnl_ocr/raw/ocr'

in_files = get_files(in_dir)

for in_file in tqdm(in_files):
    # needs to be prcessed?
    out = out_file_name(out_dir, in_file)
    if not os.path.isfile(out):
        # is there a notes file?
        notes_file = os.path.join(notes_dir, os.path.basename(in_file))
        if os.path.isfile(notes_file):
            print('processing', in_file)
            remove_notes(in_file, notes_file, out_dir)